# MODELAGEM DE DISPERSÃO

Atividade 03 - Parte 01
Nesta atividade o tópicos realizados foram:

1- Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas;

2- Implementar a função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade;

3- Implementar a função de estimativa de sobrelevação da pluma utilizando os métodos de Davidson-Bryant, Holland e Briggs. Deve ser considerado o efeito Tip-Downwash;

4- Implementar a função do modelo gaussiano;

5- Realizar simulações com o script criado, utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491;

6- Plotagem e discussão dos resultados.

1- Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas
Utilizando os dados do EZ-AERMOD de 5 dias do verão e 5 duias do inverno e determinando sua classe com base na Classificação de Estabilidade de Pasquill.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Carregar os dados
data = pd.read_csv('C:\\Users\\Dell\\OneDrive\\Documentos\\GitHub\\Polatm_RamonVieira\\inputs\\EZ_AERMOD_RamonVieira_2024-06-04\\METEO.csv')

# Ajustar o ano de dois dígitos para quatro dígitos
data['year'] = data['year'].apply(lambda x: 2000 + x if x < 100 else x)

# Criar a coluna datetime
data['datetime'] = pd.to_datetime(data[['year', 'month', 'day', 'hour']])

# Configurar o índice datetime
data.set_index('datetime', inplace=True)

# Selecionar 5 dias de verão e 5 dias de inverno
verao = data[(data.index.month >= 12) | (data.index.month <= 2)].sample(n=5, random_state=1)
inverno = data[(data.index.month >= 6) & (data.index.month <= 8)].sample(n=5, random_state=1)

# Concatenar os dias selecionados
selecionados = pd.concat([verao, inverno])

# Calcular a média horária dos parâmetros numéricos
medias_diarias = selecionados.groupby(selecionados.index.date).mean()

# Função para determinar a classe de estabilidade
def determinar_classe_estabilidade(row):
    cobertura_nuvens = row['cloud cover (tenths)']
    velocidade_vento = row['wind speed (m/s)']
    fluxo_calor_sensivel = row['sensible heat flux (W/m2)']
    
    if fluxo_calor_sensivel > 10:
        insolacao = 'forte'
    elif 3 < fluxo_calor_sensivel <= 10:
        insolacao = 'moderada'
    else:
        insolacao = 'fraca'

    if cobertura_nuvens < 3:
        if velocidade_vento < 2:
            return 'A' if insolacao == 'forte' else 'B'
        elif 2 <= velocidade_vento < 3:
            return 'B' if insolacao == 'forte' else 'C'
        else:
            return 'C'
    elif 3 <= cobertura_nuvens < 7:
        if velocidade_vento < 2:
            return 'B' if insolacao == 'forte' else 'C'
        elif 2 <= velocidade_vento < 5:
            return 'C'
        else:
            return 'D'
    else:
        if velocidade_vento < 2:
            return 'C'
        elif 2 <= velocidade_vento < 5:
            return 'D'
        else:
            return 'E'

# Aplicar a função para cada linha
medias_diarias['classe_estabilidade'] = medias_diarias.apply(determinar_classe_estabilidade, axis=1)

# Exibir os resultados
print(medias_diarias[['cloud cover (tenths)', 'wind speed (m/s)', 'sensible heat flux (W/m2)', 'classe_estabilidade']])